In [1]:
from typing import List, Tuple

from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import BaseScheduler


class Procedure(Agent):
    def __init__(
        self, model: Model, unique_id: int, daily_supply: int, initial_backlog: int
    ) -> None:
        super().__init__(unique_id, model)
        self.daily_supply = daily_supply
        self.backlog = initial_backlog

    def step(self):
        # Start by assigning procedures from preexisting global surplus.
        self.assigned = min(self.backlog, self.model.excess_supply)
        self.backlog -= self.assigned
        self.model.excess_supply -= self.assigned
        
        # Continue assigning any remaining procedures from daily supply.
        self_assigned = min(self.backlog, self.daily_supply)
        self.backlog -= self_assigned
        self.excess_supply = self.daily_supply - self_assigned
        self.assigned += self_assigned

        # Donate any excess supply back to the global procedure surplus.
        self.model.excess_supply += self.excess_supply


class Simulation(Model):
    def __init__(self, procedures: List[Tuple[int, int, int]]):
        self.excess_supply = 0
        self.schedule = BaseScheduler(self)

        # Create agents
        for procedure in procedures:
            self.schedule.add(Procedure(self, *procedure))

        self.datacollector = DataCollector(
            agent_reporters={
                "Assigned": "assigned",
                "Excess Supply": "excess_supply",
                "Pending Backlog": "backlog",
            }
        )

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [2]:
model = Simulation(
    [
        (0, 20, 5),   # Procedure A
        (1, 10, 50),  # Procedure B
    ]
)

In [3]:
model.step()
model.datacollector.get_agent_vars_dataframe()

Assigned  Excess Supply  Pending Backlog
Step AgentID                                          
1    0               5             15                0
     1              25              0               25

In [4]:
model.step()
model.datacollector.get_agent_vars_dataframe()

Assigned  Excess Supply  Pending Backlog
Step AgentID                                          
1    0               5             15                0
     1              25              0               25
2    0               0             20                0
     1              25              5                0

In [5]:
model.step()
model.datacollector.get_agent_vars_dataframe()

Assigned  Excess Supply  Pending Backlog
Step AgentID                                          
1    0               5             15                0
     1              25              0               25
2    0               0             20                0
     1              25              5                0
3    0               0             20                0
     1               0             10                0